In [31]:
import http.client
import config
import pandas as pd
import json
import folium

In [32]:
def get_arrivals(airlines, dates):
    conn = http.client.HTTPSConnection("flight-fare-search.p.rapidapi.com")
    conn2 = http.client.HTTPSConnection("flight-fare-search.p.rapidapi.com")

    headers = {
        'X-RapidAPI-Key': f"{config.api_key}",
        'X-RapidAPI-Host': "flight-fare-search.p.rapidapi.com"
    }

    df_arrivals = pd.DataFrame()  # Initialize an empty DataFrame
    df_departures = pd.DataFrame()  # Initialize an empty DataFrame

    for airline in airlines:
        for date in dates:
            conn.request("GET", f"/v2/airport/arrivals?airportCode=ATL&date={date}&carrierCode={airline}", headers=headers)
            conn2.request("GET", f"/v2/airport/departures?airportCode=ATL&date={date}&carrierCode={airline}", headers=headers)

            arrivals = conn.getresponse().read().decode('utf-8')
            departures = conn2.getresponse().read().decode('utf-8')

            arrivals = json.loads(arrivals)
            departures = json.loads(departures)

            arrivals_data = arrivals['result']['flights']
            departures_data = departures['result']['flights']

            df_arrivals_temp = pd.json_normalize(arrivals_data)  # Temporary DataFrame for each request
            df_departures_temp = pd.json_normalize(departures_data)  # Temporary DataFrame for each request

            df_arrivals = pd.concat([df_arrivals, df_arrivals_temp])  # Concatenate the temporary DataFrame with the main one

    return df_arrivals, df_departures

In [45]:
conn = http.client.HTTPSConnection("app.goflightlabs.com")
conn2 = http.client.HTTPSConnection("app.goflightlabs.com")
conn3 = http.client.HTTPSConnection("app.goflightlabs.com")
conn4 = http.client.HTTPSConnection("app.goflightlabs.com")
conn5 = http.client.HTTPSConnection("app.goflightlabs.com")

conn.request("GET", f"/flights?access_key={config.api_key}&depIata=ATL")
conn2.request("GET", f"/flights?access_key={config.api_key}&arrIata=ATL")
conn3.request("GET", f"/historical/2023-04-12?access_key={config.api_key}&code=ATL&type=departure")
conn4.request("GET", f"/historical/2023-04-12?access_key={config.api_key}&code=ATL&type=arrival")

conn5.request("GET", f"/historical/2023-04-12?access_key={config.api_key}&code=LIT&type=arrival")

real_time_departures = conn.getresponse().read().decode('utf-8')
real_time_departures = json.loads(real_time_departures)

real_time_arrivals = conn2.getresponse().read().decode('utf-8')
real_time_arrivals = json.loads(real_time_arrivals)

historical_departures = conn3.getresponse().read().decode('utf-8')
historical_departures = json.loads(historical_departures)

historical_arrivals = conn4.getresponse().read().decode('utf-8')
historical_arrivals = json.loads(historical_arrivals)

,type,status,departure.iataCode,departure.icaoCode,departure.terminal,departure.gate,departure.delay,departure.scheduledTime,departure.estimatedTime,departure.actualTime,...,flight.number,flight.iataNumber,flight.icaoNumber,codeshared.airline.name,codeshared.airline.iataCode,codeshared.airline.icaoCode,codeshared.flight.number,codeshared.flight.iataNumber,codeshared.flight.icaoNumber,arrival.delay
0,arrival,landed,dfw,kdfw,b,b40,9.0,2023-04-11t18:31:00.000,2023-04-11t18:54:00.000,2023-04-11t18:40:00.000,...,4776,as4776,asa4776,american airlines,aa,aal,3644,aa3644,aal3644,NaN
1,arrival,landed,dfw,kdfw,b,b40,9.0,2023-04-11t18:31:00.000,2023-04-11t18:54:00.000,2023-04-11t18:40:00.000,...,2376,ba2376,baw2376,american airlines,aa,aal,3644,aa3644,aal3644,NaN
2,arrival,landed,dfw,kdfw,b,b40,9.0,2023-04-11t18:31:00.000,2023-04-11t18:54:00.000,2023-04-11t18:40:00.000,...,6610,gf6610,gfa6610,american airlines,aa,aal,3644,aa3644,aal3644,NaN
3,arrival,landed,dfw,kdfw,b,b40,9.0,2023-04-11t18:31:00.000,2023-04-11t18:54:00.000,2023-04-11t18:40:00.000,...,4857,qf4857,qfa4857,american airlines,aa,aal,3644,aa3644,aal3644,NaN
4,arrival,landed,dfw,kdfw,b,b40,9.0,2023-04-11t18:31:00.000,2023-04-11t18:54:00.000,2023-04-11t18:40:00.000,...,2767,qr2767,qtr2767,american airlines,aa,aal,3644,aa3644,aal3644,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,arrival,scheduled,sdf,ksdf,NaN,NaN,27.0,2023-04-12t04:32:00.000,2023-04-12t04:59:00.000,NaN,...,722,5x722,ups722,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,arrival,unknown,qqn,kjwn,NaN,NaN,6.0,2023-04-12t15:48:00.000,2023-04-12t15:54:00.000,NaN,...,807,1i807,eja807,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,arrival,scheduled,bfm,kbfm,NaN,NaN,NaN,2023-04-12t18:13:00.000,2023-04-12t17:25:00.000,NaN,...,354,,dzr354,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,arrival,landed,qth,kgph,NaN,NaN,13.0,2023-04-11t22:30:00.000,2023-04-11t22:40:00.000,2023-04-11t22:42:00.000,...,19,,ctl19,NaN,NaN,NaN,NaN,NaN,NaN,NaN


{'success': True,
 'data': [{'type': 'departure',
   'status': 'active',
   'departure': {'iataCode': 'atl',
    'icaoCode': 'katl',
    'terminal': 's',
    'gate': 'b27',
    'delay': 13,
    'scheduledTime': '2023-04-11t20:00:00.000',
    'estimatedTime': '2023-04-11t20:00:00.000',
    'actualTime': '2023-04-11t20:12:00.000',
    'estimatedRunway': '2023-04-11t20:12:00.000',
    'actualRunway': '2023-04-11t20:12:00.000'},
   'arrival': {'iataCode': 'lit',
    'icaoCode': 'klit',
    'gate': '2',
    'scheduledTime': '2023-04-11t20:36:00.000',
    'estimatedTime': '2023-04-11t20:23:00.000'},
   'airline': {'name': 'delta air lines', 'iataCode': 'dl', 'icaoCode': 'dal'},
   'flight': {'number': '2083',
    'iataNumber': 'dl2083',
    'icaoNumber': 'dal2083'}},
  {'type': 'departure',
   'status': 'active',
   'departure': {'iataCode': 'atl',
    'icaoCode': 'katl',
    'terminal': 's',
    'gate': 'b17',
    'delay': 33,
    'scheduledTime': '2023-04-11t20:00:00.000',
    'estimatedTi

In [43]:
df_real_time_departures = pd.json_normalize(real_time_departures['data'])
df_real_time_arrials = pd.json_normalize(real_time_arrivals['data'])
df_real_time_flights = pd.concat([df_real_time_departures, df_real_time_arrials])
df_hist_departures = pd.json_normalize(historical_departures['data'])
df_hist_arrivals = pd.json_normalize(historical_arrivals['data'])
df_hist_flights = pd.concat([df_hist_departures, df_hist_arrivals])

In [46]:
df_real_time_flights

,status,aircraft.iataCode,aircraft.icao24,aircraft.icaoCode,aircraft.regNumber,airline.iataCode,airline.icaoCode,arrival.iataCode,arrival.icaoCode,departure.iataCode,...,flight.number,geography.altitude,geography.direction,geography.latitude,geography.longitude,speed.horizontal,speed.isGround,speed.vspeed,system.squawk,system.updated
0,en-route,A359,4BB0E9,A359,TC-LGI,TK,THY,IST,LTBA,ATL,...,32,11582.400,72,47.98,-45.77,1031.560,0,0,None,1684387017
1,en-route,B77W,485000,B77W,PH-BVK,KL,KLM,AMS,EHAM,ATL,...,622,10668.000,55,45.78,-64.28,1005.640,0,0,None,1684386997
2,en-route,A339,407DC5,A339,G-VTOM,VS,VIR,LHR,EGLL,ATL,...,104,11887.200,101,52.30,-7.82,837.104,0,0,None,1684386970
3,en-route,B763,A12FDA,B763,N176DZ,DL,DAL,MXP,LIMC,ATL,...,174,10972.800,100,49.50,-3.58,798.212,0,0,None,1684386989
4,unknown,A320,A3D890,A320,N347NW,DL,DAL,MDW,KMDW,ATL,...,1076,186.538,42,41.79,-87.75,72.228,0,0,None,1684386302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,en-route,B764,AB4FAE,B764,N828MH,DL,DAL,ATL,KATL,EZE,...,110,9753.600,342,5.99,-72.78,916.740,0,0,None,1684386419
20,en-route,B763,A34AA0,B763,N311AZ,,,ATL,KATL,ILN,...,3440,2659.380,157,39.26,-83.68,529.672,0,0,None,1684386659
21,en-route,A20N,A4798F,A20N,N388FR,F9,FFT,ATL,KATL,DEN,...,2300,11277.600,115,35.03,-87.85,872.292,0,0,None,1684386693
22,started,B77W,06A078,B77W,A7-BAK,QR,QTR,ATL,KATL,DOH,...,755,0.000,157,25.26,51.61,0.000,1,0,None,1684386672


In [36]:
l = ['departure.scheduledTime', 'departure.estimatedTime', 'departure.actualTime', 'departure.estimatedRunway', 'departure.actualRunway', 'arrival.scheduledTime', 'arrival.estimatedTime', 'arrival.actualTime', 'arrival.estimatedRunway', 'arrival.actualRunway']
for i in l:
    df_hist_flights[i] = pd.to_datetime(df_hist_flights[i])


df_hist_flights

,type,status,departure.iataCode,departure.icaoCode,departure.terminal,departure.gate,departure.delay,departure.scheduledTime,departure.estimatedTime,departure.actualTime,...,codeshared.airline.name,codeshared.airline.iataCode,codeshared.airline.icaoCode,codeshared.flight.number,codeshared.flight.iataNumber,codeshared.flight.icaoNumber,arrival.terminal,arrival.actualTime,arrival.estimatedRunway,arrival.actualRunway
0,departure,active,atl,katl,s,b27,13.0,2023-04-11 20:00:00,2023-04-11 20:00:00,2023-04-11 20:12:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
1,departure,active,atl,katl,s,b17,33.0,2023-04-11 20:00:00,2023-04-11 20:12:00,2023-04-11 20:33:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
2,departure,active,atl,katl,s,b19,6.0,2023-04-11 20:00:00,2023-04-11 20:05:00,2023-04-11 20:05:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
3,departure,active,atl,katl,n,d23,30.0,2023-04-11 20:05:00,2023-04-11 20:46:00,2023-04-11 20:35:00,...,american airlines,aa,aal,5339,aa5339,aal5339,NaN,NaT,NaT,NaT
4,departure,active,atl,katl,n,d23,30.0,2023-04-11 20:05:00,2023-04-11 20:46:00,2023-04-11 20:35:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3676,arrival,landed,mem,kmem,NaN,NaN,8.0,2023-04-12 15:23:00,NaT,2023-04-12 15:30:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-12 17:21:00,2023-04-12 17:21:00,2023-04-12 17:21:00
3677,arrival,landed,cvg,kcvg,NaN,NaN,NaN,2023-04-12 17:30:00,NaT,2023-04-12 17:28:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-12 18:33:00,2023-04-12 18:33:00,2023-04-12 18:33:00
3678,arrival,unknown,crw,kcrw,NaN,NaN,21.0,2023-04-12 17:30:00,2023-04-12 17:51:00,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT
3679,arrival,landed,dfw,kdfw,NaN,NaN,3.0,2023-04-12 16:25:00,NaT,2023-04-12 16:28:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-04-12 19:07:00,2023-04-12 19:07:00,2023-04-12 19:07:00


In [37]:
# df_departures.rename(columns={'time':'Time_UTC',
#                               'operator':'Operator',
#                               'departure.time.timeAMPM':'Departure_Time_AM/PM',
#                               'departure.time.time24':'Departure_Time_24',
#                               'departure.airport.iata':'Arrival_Airport_IATA',
#                               'departure.airport.city':'Arrival_Airport_City',
#                               'arrival.timeAMPM':'Arrival_Time_AM/PM',
#                               'arrival.time24':'Arrival_Time_24',
#                               'carrier.code':'Airline_Code',
#                               'carrier.name':'Airline_Name',
#                               'carrier.flightNumber':'Airline_Flight_Number',
#                               'date':'Date_UTC',
#                               'time_EST':'Time_EST',
#                               'date_EST':'Date_EST'}, inplace=True)
#
# df_departures = df_departures[['Time_UTC', 'Date_UTC', 'Time_EST', 'Date_EST', 'Departure_Time_AM/PM', 'Departure_Time_24', 'Arrival_Airport_IATA', 'Arrival_Airport_City', 'Arrival_Time_AM/PM', 'Arrival_Time_24', 'Airline_Code', 'Airline_Name', 'Airline_Flight_Number']]

In [38]:
# df_arrivals.rename(columns={'time':'Time_UTC',
#                               'operator':'Operator',
#                               'departure.time.timeAMPM':'Departure_Time_AM/PM',
#                               'departure.time.time24':'Departure_Time_24',
#                               'departure.airport.iata':'Departure_Airport_IATA',
#                               'departure.airport.city':'Departure_Airport_City',
#                               'arrival.timeAMPM':'Arrival_Time_AM/PM',
#                               'arrival.time24':'Arrival_Time_24',
#                               'carrier.code':'Airline_Code',
#                               'carrier.name':'Airline_Name',
#                               'carrier.flightNumber':'Airline_Flight_Number',
#                               'date':'Date_UTC',
#                               'time_EST':'Time_EST',
#                               'date_EST':'Date_EST'}, inplace=True)
#
# df_arrivals = df_arrivals[['Time_UTC', 'Date_UTC', 'Time_EST', 'Date_EST', 'Departure_Time_AM/PM', 'Departure_Time_24', 'Departure_Airport_IATA', 'Departure_Airport_City', 'Arrival_Time_AM/PM', 'Arrival_Time_24', 'Airline_Code', 'Airline_Name', 'Airline_Flight_Number']]